In [9]:
import pandas as pd

In [10]:
games=pd.read_csv('games.csv')

In [11]:
games['GAME_DATE_EST']=pd.to_datetime(games['GAME_DATE_EST'])

In [12]:
games = games[games['GAME_DATE_EST'].dt.strftime('%Y') > '2017']

In [13]:
games_details=pd.read_csv('games_details.csv', low_memory=False)

In [14]:
players=pd.read_csv('players.csv')

In [15]:
ranking=pd.read_csv('ranking.csv')

In [16]:
ranking['STANDINGSDATE']=pd.to_datetime(ranking['STANDINGSDATE'])

In [17]:
ranking = ranking[ranking['STANDINGSDATE'].dt.strftime('%Y') > '2017']

In [18]:
teams=pd.read_csv('teams.csv')

In [19]:
def column_to_lowercase(df):
    lower_list=[]
    for i in range(len(list(df.columns))):
        lower_list.append(list(df.columns)[i].lower())
    df.columns=lower_list
    return df

## player 

In [20]:
player=players[["PLAYER_ID","PLAYER_NAME"]].drop_duplicates()

In [21]:
player_temp1=games_details[["PLAYER_ID","PLAYER_NAME","NICKNAME"]].drop_duplicates()

In [22]:
player=player.merge(player_temp1,how='left',left_index=True, right_index=True, suffixes=('', '_drop'))

In [23]:
player.drop([col for col in player.columns if 'drop' in col], axis=1, inplace=True)

In [24]:
player=column_to_lowercase(player)

In [25]:
player=player.rename(columns={"nickname":"player_nickname"})

## team

In [26]:
team=teams[["TEAM_ID",'MIN_YEAR',
       'ABBREVIATION','NICKNAME','YEARFOUNDED','OWNER','GENERALMANAGER','HEADCOACH','CITY']]

In [27]:
team_temp1=ranking[['TEAM_ID','CONFERENCE']].drop_duplicates()

In [28]:
team=team.merge(team_temp1,how='left',on='TEAM_ID')

In [29]:
team=column_to_lowercase(team)

In [30]:
team=team.rename(columns={"nickname":"team_nickname", "city":"team_city"})

## rankings

In [31]:
rankings=ranking[['STANDINGSDATE','TEAM_ID','G','W','L','HOME_RECORD','ROAD_RECORD','W_PCT']]
rankings=column_to_lowercase(rankings)

In [32]:
rankings=rankings.rename(columns={'standingsdate':'standingdate'})

In [33]:
ranking=rankings

In [34]:
ranking=ranking.rename(columns={'g':'number_of_games','w':'winning_games','l':'losing_games'})
ranking=ranking.drop_duplicates(subset=['standingdate', 'team_id'], keep='first')

## player_team

In [35]:
temp1=games[['GAME_ID','SEASON']].drop_duplicates()

In [36]:
temp2=games_details[['GAME_ID','PLAYER_ID','TEAM_ID']].drop_duplicates()

In [37]:
player_team=temp2.merge(temp1,how='left',on='GAME_ID')[['SEASON','PLAYER_ID','TEAM_ID']]

In [38]:
player_team=column_to_lowercase(player_team)

In [39]:
player_team=player_team.dropna()

## player justification

In [40]:
player=player.merge(player_team,how="right",on='player_id')[['player_id','player_name','player_nickname']]

In [41]:
player=player.drop_duplicates()

## player_team justification

In [42]:
player_team=player_team.drop_duplicates(subset=['season', 'player_id'], keep='first') ## drop mistakes recorded

## game

In [43]:
game=games[['GAME_ID','GAME_DATE_EST','SEASON','HOME_TEAM_ID','VISITOR_TEAM_ID','HOME_TEAM_WINS']]

In [44]:
location_temp=teams[['TEAM_ID','ARENA','ARENACAPACITY']]

In [45]:
location_temp.insert(0, 'arena_id', range(1, 1 + len(location_temp)))

In [46]:
location_temp=location_temp.rename(columns={'TEAM_ID':'HOME_TEAM_ID'}) ## team_id is same as home_team_id

In [47]:
game=game.merge(location_temp,how='left',on='HOME_TEAM_ID').drop(['ARENA','ARENACAPACITY'],axis=1)

In [48]:
game=column_to_lowercase(game)

## game_details & games_player_teams

In [49]:
games_details=column_to_lowercase(games_details)

In [50]:
game_details=game[['game_id']].merge(games_details,how='left',on='game_id')

In [51]:
game_details=game_details.drop_duplicates()

In [52]:
games_player_teams=game_details[['game_id','team_id','player_id']]

In [53]:
game_details=game_details.drop(['team_id','team_abbreviation','team_city','player_name','nickname','min'],axis=1)

In [54]:
game_details=game_details.rename(columns={'game_id':'games_id'})

In [55]:
games_player_teams=games_player_teams.rename(columns={'game_id':'games_id'}).drop_duplicates()

## game_location_info

In [56]:
game_location_info=location_temp.drop('HOME_TEAM_ID',axis=1)

In [57]:
game_location_info=column_to_lowercase(game_location_info)

## games_visitorteam / games_hometeam

In [58]:
games_visitorteam=games[['GAME_ID','VISITOR_TEAM_ID','PTS_away','FG_PCT_away','FT_PCT_away','AST_away','REB_away']]

In [59]:
games_hometeam=games[['GAME_ID','HOME_TEAM_ID','PTS_home','FG_PCT_home','FT_PCT_home','AST_home','REB_home']]

In [60]:
games_hometeam.columns=['game_id','home_team_id','PTS','FG_PCT','FT_PCT','AST','REB']

In [61]:
games_hometeam=column_to_lowercase(games_hometeam)

In [62]:
games_visitorteam.columns=['game_id','visitor_team_id','PTS','FG_PCT','FT_PCT','AST','REB']

In [63]:
games_visitorteam=column_to_lowercase(games_visitorteam)

## Details Justification to ensure the consistency

In [64]:
game=game[["game_id","game_date_est","season","home_team_wins","arena_id"]]

In [65]:
games=game

In [66]:
game_details=game_details.rename(columns={"game_id":"games_id",'to':'num_to'}).drop_duplicates(subset=['games_id', 'player_id'], keep='first')

In [67]:
games=games.rename(columns={"game_id":"games_id"})

In [68]:
games_visitorteam=games_visitorteam.rename(columns={"game_id":"games_id"}).drop_duplicates(subset=['games_id', 'visitor_team_id'], keep='first')

In [69]:
games_hometeam=games_hometeam.rename(columns={"game_id":"games_id"}).drop_duplicates()

In [70]:
games_hometeam=games_hometeam.drop_duplicates(subset=['games_id', 'home_team_id'], keep='first')

In [71]:
games=games.drop_duplicates()

# ETL

In [3]:
import sqlalchemy as sa
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import psycopg2

In [201]:
hostname = 'pgsql9.c0ttp4tkwo66.us-east-1.rds.amazonaws.com'
username = 'pgsql9'
password = 'columbia9'
database = 'pgsql9'


In [202]:
conn_url = 'postgresql://' + username + ':' + password + '@' + hostname

# create the db engine
engine = create_engine(conn_url)

# add a conenction
connection = engine.connect()

## Create Table

In [5]:
cmd="""
Drop Table Game_Details, Ranking,Player_Team, Games_Player_Teams,Games_HomeTeam,Games_VisitorTeam,Games,game_location_info,Player,Team
"""

In [6]:
cur.execute(cmd)
conn.commit()

In [160]:
cmd="""
CREATE TABLE game_location_info (
Arena_id int4 Primary Key,
Arena varchar(30),
Arenacapacity numeric(10,2)
);


CREATE TABLE games(
Games_id int8 PRIMARY KEY,
Game_date_est date,
Season int4,
Home_team_wins char(1),
Arena_id int4 NOT NULL REFERENCES game_location_info(Arena_id)
);

CREATE TABLE games_hometeam (
Games_id int8,
Home_team_id int8,
PTS int4,
FG_PCT numeric(4,3),
FT_PCT numeric(4,3),
FG3_PCT numeric(4,3),
AST int2,
REB int2,
PRIMARY KEY (Games_id, Home_team_id),
FOREIGN KEY (Games_id) references games
);

CREATE TABLE games_visitorteam (
Games_id int8,
Visitor_team_id int8,
PTS int4,
FG_PCT numeric(4,3),
FT_PCT numeric(4,3),
FG3_PCT numeric(4,3),
AST int2,
REB int2,
PRIMARY KEY (Games_id, Visitor_team_id),
FOREIGN KEY (games_id) references games
);


CREATE TABLE player
(Player_id varchar(10),
 Player_name varchar(50),
 Player_Nickname varchar(50),
 PRIMARY KEY (Player_id)
);

Create Table team(
 Team_id INT8,
 min_year INT,
 Abbreviation varchar(3),
 Team_nickname varchar(20),
 Yearfounded int,
 Owner varchar(50),
 Generalmanager varchar(50),
 Headcoach varchar(50),
 Conference varchar(4),
 Team_city varchar(20),
 PRIMARY KEY (Team_id)
);

CREATE Table games_player_teams(
Games_id int8,
Team_id int8,
Player_id varchar(10),
PRIMARY KEY (Games_id, Team_id, Player_id),
FOREIGN KEY (Games_id) references games,
FOREIGN KEY (Team_id) references team,
FOREIGN KEY (Player_id) references player
);



CREATE TABLE player_team
(Season INT,
 Player_id varchar(10),
 Team_id INT8,
 PRIMARY KEY (Season, Player_id),
 FOREIGN KEY (Team_id) references team,
 FOREIGN KEY (Player_id) references player
);

CREATE TABLE ranking
(Standingdate date,
 Team_id INT8 ,
 Number_of_Games INT,
 Winning_Games INT,
 Losing_Games INT,
 Home_record varchar(10),
 Road_record varchar(10),
 W_PCT numeric(4,3),
 PRIMARY KEY (Standingdate, Team_id),
 FOREIGN KEY (Team_id) References Team);

CREATE Table game_details(
Games_id int8,
Player_id varchar(10),
Start_position char(1),
Comment varchar(50),
FGM int,
FGA int,
FG_PCT numeric(4,3),
FG3M int,
FG3A int,
FG3_PCT numeric(4,3),
FTM int,
FTA int,
FT_PCT numeric(4,3),
OREB INT,
DREB INT,
REB INT,
AST INT,
STL INT,
BLK INT,
num_To INT,
PF INT,
PTS INT,
Plus_minus int,
PRIMARY KEY (Games_id,  Player_id),
FOREIGN KEY (Games_id) references Games,
FOREIGN KEY (Player_id) references Player
);

"""



In [161]:
connection.execute(cmd)

## Import Data

In [162]:
game_location_info.to_sql(name="game_location_info", con=engine, if_exists='append', index=False)

In [163]:
games.to_sql(name="games", con=engine, if_exists='append', index=False)

In [164]:
games_hometeam.to_sql(name="games_hometeam", con=engine, if_exists='append', index=False)

In [165]:
games_visitorteam.to_sql(name="games_visitorteam", con=engine, if_exists='append', index=False)

In [166]:
player.to_sql(name="player", con=engine, if_exists='append', index=False)

In [167]:
team.to_sql(name="team", con=engine, if_exists='append', index=False)

In [168]:
player_team.to_sql(name="player_team", con=engine, if_exists='append', index=False)

In [169]:
games_player_teams.to_sql(name="games_player_teams", con=engine, if_exists='append', index=False)

In [170]:
ranking.to_sql(name="ranking", con=engine, if_exists='append', index=False)

In [171]:
game_details.to_sql(name="game_details", con=engine, if_exists='append', index=False)

## 10 procedures

### 1. Who are top 100 players that have the highest total scores (from 2018-2022)?

In [172]:
cmd1 = """
create or replace view top100_by_score
as
select *
from
(select *, dense_rank() over (order by total_score DESC) as total_rank
from
(select player_id, p.player_name, sum(PTS) total_score
from game_details 
join player p
using (player_id)
group by player_id,player_name
having sum(PF) is not NUll) sub1
order by total_rank) sub2
where total_rank <= 100;
 
select * from top100_by_score

"""

In [173]:
result1 = connection.execute(cmd1).fetchall()

In [174]:
column_names = result1[0].keys()
result1=pd.DataFrame(result1, columns=column_names)
result1

,player_id,player_name,total_score,total_rank
0,201935,James Harden,10205,1
1,203507,Giannis Antetokounmpo,10105,2
2,203081,Damian Lillard,8851,3
3,203999,Nikola Jokic,8771,4
4,2544,LeBron James,8728,5
...,...,...,...,...
96,200755,JJ Redick,3737,96
97,1627854,Bryn Forbes,3736,97
98,201143,Al Horford,3714,98
99,202691,Klay Thompson,3700,99


### 2. What is the number of total games that each player played in each season, and the number of games that a player gets 3-pointers in each season?


In [175]:
cmd2 = """
create or replace view count_of_3point
as
(select season, player_id, count(games_id) count_3point
from
(select games_id, player_id, fg3m, season
from game_details
join games
using(games_id)
where fg3m>0) point_game
group by season, player_id
order by season, count_3point DESC);
 
select * from count_of_3point;
"""

In [203]:
cmd2_1="""
create or replace view count_of_total
as
(select season, player_id, count(games_id) total_games
from
(select g.season, gpt.games_id, gpt.player_id
from games_player_teams gpt
join games g
using(games_id)) sub1
group by season, player_id
order by season, total_games DESC);
 
select * from count_of_total;
"""

In [204]:
result2_1 = connection.execute(cmd2_1).fetchall()
column_names = result2_1[0].keys()
result2_1=pd.DataFrame(result2_1, columns=column_names)
result2_1

,season,player_id,total_games
0,2017,203903,68
1,2017,2544,68
2,2017,2747,67
3,2017,2585,66
4,2017,201156,66
...,...,...,...
3052,2021,1630777,1
3053,2021,1630791,1
3054,2021,1630672,1
3055,2021,1629017,1


In [178]:
result2 = connection.execute(cmd2).fetchall()
column_names = result2[0].keys()
result2=pd.DataFrame(result2, columns=column_names)
result2

,season,player_id,count_3point
0,2017,202691,54
1,2017,204060,53
2,2017,1626179,53
3,2017,203496,52
4,2017,1628378,52
...,...,...,...
2440,2021,1629619,1
2441,2021,203920,1
2442,2021,1627782,1
2443,2021,1629686,1


### 3. What is the probability of games that players made 3-pointers compared to the total games he played in each season?

In [179]:
cmd3 = """
create or replace view percentage_of_3point
as
(select season, player_id, player_name, round((count_3point* 100.0 /total_games),2) as avg_3point
from
(select count_of_3point.season, count_of_3point.player_id, total_games, count_3point
from count_of_total
left join count_of_3point
using(season,player_id)
order by season) sub1
join player
using(player_id)
order by season, avg_3point DESC);
 
select * from percentage_of_3point;
"""

In [180]:
result3 = connection.execute(cmd3).fetchall()
column_names = result3[0].keys()
result3=pd.DataFrame(result3, columns=column_names)
result3

,season,player_id,player_name,avg_3point
0,2017,201281,Andre Ingram,100.00
1,2017,203081,Damian Lillard,97.83
2,2017,201939,Stephen Curry,97.62
3,2017,200768,Kyle Lowry,96.30
4,2017,200755,JJ Redick,95.92
...,...,...,...,...
2440,2021,1629646,None,1.56
2441,2021,202684,Tristan Thompson,1.49
2442,2021,2617,Udonis Haslem,1.47
2443,2021,1630233,None,1.47


### 4. Which players in season 2021 in the top 100 list are also the players who wins 3-pointers in more than 90% of games he joined?(with their rankings in top100 list)?

In [181]:
cmd4 = """
select DISTINCT top100_by_score.player_name, team_nickname, total_rank
from percentage_of_3point
join top100_by_score
using(player_id)
join player_team
using (player_id)
join team 
using(team_id)
where avg_3point>90 and player_id in (select player_id from top100_by_score) and player_team.season=2021
order by total_rank
"""

In [182]:
result4 = connection.execute(cmd4).fetchall()
column_names = result4[0].keys()
result4=pd.DataFrame(result4, columns=column_names)
result4

,player_name,team_nickname,total_rank
0,James Harden,76ers,1
1,Damian Lillard,Trail Blazers,3
2,Donovan Mitchell,Jazz,6
3,Bradley Beal,Wizards,7
4,Jayson Tatum,Celtics,9
5,Stephen Curry,Warriors,12
6,Paul George,Clippers,15
7,Trae Young,Hawks,16
8,Luka Doncic,Mavericks,17
9,CJ McCollum,Pelicans,19


### 5. For player who is changing teams, in which team did he score the most points？

In [183]:
cmd5 = """
create or replace view score_by_team
as
(select player_id,team_id,sum(PF) as total_score
from game_details 
join games_player_teams
using(games_id,player_id)
where player_id in 
(select player_id 
from
(select player_id, count(team_id) count_of_change
from player_team
group by player_id 
having count(team_id) >1) sub1)
group by (player_id, team_id)
order by total_score DESC,player_id);
 
select player_name, team_nickname,total_score
from score_by_team
join player
using (player_id)
join team
using(team_id)
where (player_id,total_score) in
(select player_id,max(total_score) from score_by_team
group by player_id)
order by total_score DESC;
"""

In [184]:
result5 = connection.execute(cmd5).fetchall()
column_names = result5[0].keys()
result5=pd.DataFrame(result5, columns=column_names)
result5

,player_name,team_nickname,total_score
0,Nikola Jokic,Nuggets,1127
1,Giannis Antetokounmpo,Bucks,1115
2,Pascal Siakam,Raptors,1029
3,Karl-Anthony Towns,Timberwolves,1025
4,Rudy Gobert,Jazz,1016
...,...,...,...
772,Donte Grantham,Clippers,0
773,Kaiser Gates,Bulls,0
774,Kaiser Gates,Celtics,0
775,None,Hornets,0


### 6. Based on the conference (team), which is the best players to represent the east and west?

In [211]:
cmd6 = """
create or replace view west
as
(select top100_by_score.player_id, top100_by_score.player_name, 
max(top100_by_score.total_score) as total_scores, conference
from top100_by_score, player_team, team
where top100_by_score.player_id = player_team.player_id
and player_team.team_id = team.team_id
and conference = 'West'
and season = 2021
group by top100_by_score.player_id, top100_by_score.player_name, conference
order by total_scores desc
limit 1);
 
create or replace view east
as
(select top100_by_score.player_id, top100_by_score.player_name, 
max(top100_by_score.total_score) as total_scores, conference
from top100_by_score, player_team, team
where top100_by_score.player_id = player_team.player_id
and player_team.team_id = team.team_id
and conference = 'East'
and season = 2021
group by top100_by_score.player_id, top100_by_score.player_name, conference
order by total_scores desc
limit 1);
 
select *
from west
union
select *
from east
"""

In [212]:
result6 = connection.execute(cmd6).fetchall()
column_names = result6[0].keys()
result6=pd.DataFrame(result6, columns=column_names)
result6

,player_id,player_name,total_scores,conference
0,201935,James Harden,10205,East
1,203081,Damian Lillard,8851,West


### 7.Which is the high-risk player that has the highest PF?

In [213]:
cmd7="""
select p.player_id, p.player_name, sum(pf) as total_pf
from game_details gd, player p
where gd.player_id = p.player_id
group by p.player_id, p.player_name
having sum(pf) is not NULL
order by total_pf desc
limit 1
"""

In [214]:
result7 = connection.execute(cmd7).fetchall()
column_names = result7[0].keys()
result7=pd.DataFrame(result7, columns=column_names)
result7

,player_id,player_name,total_pf
0,203999,Nikola Jokic,1127


### 8. Which team won the highest number of games when away, and who is the owner?

In [215]:
cmd8="""
select t.team_id, t.team_nickname, owner, count(*) as winning_away
from games_visitorteam gv, team t, games g
where gv.visitor_team_id = t.team_id
and g.games_id = gv.games_id
and home_team_wins = '0'
group by t.team_id, t.team_nickname, owner
order by winning_away desc
limit 1
"""

In [216]:
result8 = connection.execute(cmd8).fetchall()
column_names = result8[0].keys()
result8=pd.DataFrame(result8, columns=column_names)
result8

,team_id,team_nickname,owner,winning_away
0,1610612749,Bucks,Wesley Edens & Marc Lasry,121


### 9. Which team has won the most games in the last five years, and how many games have they won in total?

In [217]:
cmd9="""
create or replace view win_game
as
(select games_id, home_team_wins,home_team_id, visitor_team_id
from games
join games_hometeam
using(games_id)
join games_visitorteam
using(games_id));
 
select team_id, team_nickname, sum(count_of_win) total_wins
from 
(select home_team_id team_id, count(games_id) count_of_win
from win_game
where home_team_wins = '1'
group by home_team_id
union
select visitor_team_id team_id, count(games_id) count_of_win
from win_game
where home_team_wins = '0'
group by visitor_team_id) sub1
join team
using(team_id)
group by team_id, team_nickname
order by total_wins DESC
limit 1;
"""

In [218]:
result9 = connection.execute(cmd9).fetchall()
column_names = result9[0].keys()
result9=pd.DataFrame(result9, columns=column_names)
result9

,team_id,team_nickname,total_wins
0,1610612749,Bucks,274


### 10. Which arena has won the most games over the past five years?

In [219]:
cmd10="""
create or replace view win_by_arena
as
(select home_team_id team_id, arena_id, count(games_id) count_of_win
from win_game
join games
using(games_id)
where win_game.home_team_wins = '1'
group by home_team_id,arena_id
union
select visitor_team_id team_id, arena_id, count(games_id) count_of_win
from win_game
join games
using(games_id)
where win_game.home_team_wins = '0'
group by visitor_team_id,arena_id);
 
select arena, sum(count_of_win) total_win_by_arena
from win_by_arena
join game_location_info
using(arena_id)
group by arena
order by total_win_by_arena DESC
limit 1
"""

In [221]:
result10 = connection.execute(cmd10).fetchall()
column_names = result10[0].keys()
result10=pd.DataFrame(result10, columns=column_names)
result10

,arena,total_win_by_arena
0,Staples Center,405


In [222]:
connection.close()